In [23]:
import numpy as np 
from matplotlib import pyplot as plt 
import pandas as pd

In [24]:
data =  pd.read_csv('all_data_compiled.csv')
print(data)

      index_compiled  index  class  B_B_FLUX_AP  B_B_FLUX_AP_HI  \
0                  0     89    1.0     0.000174        0.000182   
1                  1    370    1.0     0.000462        0.000477   
2                  2    645    0.0     0.000003        0.000003   
3                  3    275    1.0     0.000002        0.000002   
4                  4    474    0.0     0.000002        0.000002   
...              ...    ...    ...          ...             ...   
1466            1466    221    1.0     0.000002        0.000002   
1467            1467    864    0.0     0.000000        0.000000   
1468            1468    777    0.0     0.000080        0.000084   
1469            1469    831    0.0     0.000000        0.000000   
1470            1470    374    1.0     0.001739        0.001878   

      B_B_FLUX_AP_LO  B_M_FLUX_AP  B_M_FLUX_AP_HI  B_M_FLUX_AP_LO  \
0           0.000166     0.000569        0.000616        0.000526   
1           0.000444     0.005532        0.005737        

In [25]:
print(data.shape)
x = data.drop(['index_compiled', 'index', 'class'],axis=1)
print(x.shape)
y = data['class']
y.shape

(1471, 364)
(1471, 361)


(1471,)

In [26]:
x_np =  x.to_numpy()
y_np =  y.to_numpy()
x_train = x_np[:1000,:]
y_train = y_np[:1000]
print(x_train.shape)
print(y_train.shape)

(1000, 361)
(1000,)


In [29]:
def sparsity(mat):
    m,n =  mat.shape
    count = m*n
    for i in range(m):
        for j in range(n):
            if(mat[i,j]>0):
                count-=1
    return count/(m*n)
spars = sparsity(x_np)
print('Data Sparsity :{:.2f}'.format(spars))

Data Sparsity :0.52


In [5]:
def split_data(x,y,s):   
    split = 0.7 
    split_no = int(split*len(y))
    x_train = x[:split_no, :]
    y_train = y[:split_no]
    x_test = x[split_no:, :]
    y_test = y[split_no:]
    return ((x_train,y_train) , (x_test ,y_test))

(x_train,y_train),(x_test,y_test) = split_data(x_np, y_np, 0.8)
print(x_train.shape ,y_train.shape)
print(x_test.shape ,y_test.shape)

(1029, 361) (1029,)
(442, 361) (442,)


# Neural Netork starts here

In [6]:
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

In [7]:
one_hot_y_train =  to_categorical(y_train)
one_hot_y_test =  to_categorical(y_test)

In [8]:
def model_gen(shape):

    inputs =  keras.Input(shape=(361,))
    dense =  layers.Dense(64, activation='relu')
    x = dense(inputs)
    x =  layers.BatchNormalization(axis=-1)(x)
    #x = layers.Dropout(0.3)(x)
    for s in shape:
        x = layers.Dense(64, activation='relu')(x)
    
    outputs = layers.Dense(2 , activation='sigmoid')(x)
    model = keras.Model(inputs=inputs , outputs=outputs , name='trial_model')
    model.compile(
        loss = "categorical_crossentropy",
        optimizer = keras.optimizers.Adam(),
        metrics = ["accuracy"],
    )
    return model
model = model_gen([64,64,32,8])
history = model.fit(x_train, one_hot_y_train, batch_size=128, epochs=40, validation_split=0.1)

Epoch 1/40
8/8 [==============================] - 1s 43ms/step - loss: 0.6060 - accuracy: 0.7106 - val_loss: 0.6558 - val_accuracy: 0.6602
Epoch 2/40
8/8 [==============================] - 0s 7ms/step - loss: 0.4873 - accuracy: 0.7732 - val_loss: 0.6172 - val_accuracy: 0.7379
Epoch 3/40
8/8 [==============================] - 0s 7ms/step - loss: 0.4267 - accuracy: 0.8121 - val_loss: 0.5999 - val_accuracy: 0.6990
Epoch 4/40
8/8 [==============================] - 0s 7ms/step - loss: 0.3774 - accuracy: 0.8305 - val_loss: 0.6010 - val_accuracy: 0.7573
Epoch 5/40
8/8 [==============================] - 0s 8ms/step - loss: 0.3424 - accuracy: 0.8650 - val_loss: 0.5863 - val_accuracy: 0.7282
Epoch 6/40
8/8 [==============================] - 0s 8ms/step - loss: 0.3096 - accuracy: 0.8737 - val_loss: 0.5690 - val_accuracy: 0.7961
Epoch 7/40
8/8 [==============================] - 0s 8ms/step - loss: 0.2823 - accuracy: 0.8769 - val_loss: 0.5644 - val_accuracy: 0.7282
Epoch 8/40
8/8 [=================

In [20]:
def acc_score(model , y_test , x_test):
    y_pred = model.predict(x_test)
    y_pred = [np.argmax(r) for r in y_pred]
    y_test = [np.argmax(r) for r in y_test]
    #print(y_pred)
    correct = 0
    total =  len(y_test)
    for y1,y2 in zip(y_pred,y_test):
        if(int(y1)==int(y2)):
            correct+=1
        else:
            continue
    print('correct prediction :' , correct)
    print('total prediction :' , total)
    print('score: ' , correct/total*100)
    
print('training data prediction')
acc_score(model, one_hot_y_train, x_train)
print('----------------------------------')
print('test data prediciton')
acc_score(model, one_hot_y_test, x_test)

training data prediction
correct prediction : 964
total prediction : 1029
score:  93.68318756073857
----------------------------------
test data prediciton
correct prediction : 388
total prediction : 442
score:  87.78280542986425


In [19]:
res = model.predict(x_test)
res = [np.argmax(r) for r in res]
#print(res)
count = 0
for a,b in zip(res,y_test):
    if(int(a)==int(b)):
        count+=1
print(count , len(y_test))
print(count/len(y_test))

388 442
0.8778280542986425
